In [4]:
# Import the required libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load and preprocess the time series data
data = pd.read_csv('TSLA.csv')
# Perform data preprocessing steps (e.g., handle missing values, normalization)

# Split the data into training and testing sets
train_size = int(len(data) * 0.8)
train_data, test_data = data[:train_size], data[train_size:]

# Prepare the input and output sequences
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(X), np.array(y)

sequence_length = 10
X_train, y_train = create_sequences(train_data, sequence_length)
X_test, y_test = create_sequences(test_data, sequence_length)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=64, input_shape=(sequence_length, 1)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Predict on the test set
predictions = model.predict(X_test)

# Evaluate the model
mse = np.mean((predictions - y_test) ** 2)
print("Mean Squared Error:", mse)


KeyError: 10

In [ ]:
#Dane początkowe
data = daily_visits.copy()
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

import numpy as np
from keras.models import Sequential
from keras.layers import  LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


#Dane do treningu modelu
df = data
y = data['No_visits']
y = y.values.reshape(-1, 1)

# skalowanie danych
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(y)
y = scaler.transform(y)


n_lookback = 150  # dane na których będzie się uczył model
n_forecast = 30  # długość prognozy

X = []
Y = []

for i in range(n_lookback, len(y) - n_forecast + 1):
    X.append(y[i - n_lookback: i])
    Y.append(y[i: i + n_forecast])

X = np.array(X)
Y = np.array(Y)

# budowa modelu
model = Sequential()
model.add(LSTM(units=32, return_sequences=True, input_shape=(n_lookback, 1)))
model.add(LSTM(units=32))
model.add(Dense(n_forecast))



model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, Y, epochs=100, batch_size=8, verbose=0)

#prognozowanie
X_ = y[- n_lookback:]  
X_ = X_.reshape(1, n_lookback, 1)

Y_ = model.predict(X_).reshape(-1, 1)
Y_ = scaler.inverse_transform(Y_)

# zapis wyników
df_past = df[['No_visits']].reset_index()
df_past.rename(columns={'index': 'Date', 'No_visits': 'Actual'}, inplace=True)
df_past['Date'] = pd.to_datetime(df_past['Date'])
df_past['Forecast'] = np.nan
df_past['Forecast'].iloc[-1] = df_past['Actual'].iloc[-1]

df_future = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_future['Date'] = pd.date_range(start=df_past['Date'].iloc[-1] + pd.Timedelta(days=1), periods=n_forecast)
df_future['Forecast'] = Y_.flatten()
df_future['Actual'] = np.nan

results = df_past.append(df_future).set_index('Date')

# wykres
results.plot(title='Prognoza wizyt w lipcu')

In [ ]:
def Sequential_Input_LSTM(df, input_sequence):
    df_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df_np) - input_sequence):
        row = [a for a in df_np[i:i + input_sequence]]
        X.append(row)
        label = df_np[i + input_sequence]
        y.append(label)
        
    return np.array(X), np.array(y)


n_input = 10      

df_min_model_data = df_hour_lvl['T']

X, y = Sequential_Input_LSTM(df_min_model_data, n_input)

In [ ]:
# Training data
X_train, y_train = X[:60000], y[:60000]

# Validation data
X_val, y_val = X[60000:65000], y[60000:65000]

# Test data
X_test, y_test = X[65000:], y[65000:]

In [ ]:
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam 

In [ ]:
n_features = 1                        

model1 = Sequential()

model1.add(InputLayer((n_input,n_features)))
model1.add(LSTM(100, return_sequences = True))     
model1.add(LSTM(100, return_sequences = True))
model1.add(LSTM(50))
model1.add(Dense(8, activation = 'relu'))
model1.add(Dense(1, activation = 'linear'))

model1.summary()

In [ ]:
losses_df1 = pd.DataFrame(model1.history.history)

losses_df1.plot(figsize = (10,6))

In [ ]:
# save the model
save_model(model1, "LSTM_Models/lstm_univariate.h5")

# load the model
model1 = load_model('LSTM_Models/lstm_univariate.h5')

In [ ]:
test_predictions1 = model1.predict(X_test).flatten()


X_test_list = []
for i in range(len(X_test)):
    X_test_list.append(X_test[i][0])
    

test_predictions_df1 = pd.DataFrame({'X_test':list(X_test_list), 
                                    'LSTM Prediction':list(test_predictions1)})

In [ ]:
test_predictions_df1.plot(figsize = (15,6))

In [ ]:
test_predictions_df1[(len(X_test) - 720):].plot(figsize = (15,5))

In [ ]:
def futureForecast(df, col, n_input, n_features, forecast_timeperiod, model):

    x_input = np.array(df[len(df)-n_input:][col])

    temp_input=list(x_input)

    lst_output=[]
    i=0

    while(i < forecast_timeperiod):

        if(len(temp_input) > n_input):

            x_input = np.array(temp_input[1:])
            x_input = x_input.reshape((1, n_input, n_features))
            yhat = model.predict(x_input, verbose=0)
            temp_input.append(yhat[0][0])
            temp_input = temp_input[1:]
            lst_output.append(yhat[0][0])

            i=i+1

        else:
            x_input = x_input.reshape((1, n_input, n_features))
            yhat = model.predict(x_input, verbose=0)
            #print(yhat[0])
            temp_input.append(yhat[0][0])
            lst_output.append(yhat[0][0])

            i=i+1
            
    return lst_output

In [ ]:
n_input = 10
n_features = 1
forecast_timeperiod = 240         # next 10 days
model = model1

forecast_output = futureForecast(df, 
                                 'T', 
                                 n_input, 
                                 n_features, 
                                 forecast_timeperiod, 
                                 model)

In [ ]:
last_10_days = df['T'][len(df) - 240:].tolist()

next_10_days = pd.DataFrame(forecast_output, columns = ['FutureForecast'])

In [ ]:
plt.figure(figsize = (15,5))

hist_axis = len(last_10_days)
forecast_axis = hist_axis + len(next_10_days)

plt.plot(np.arange(0,hist_axis),last_10_days, color = 'blue')
plt.plot(np.arange(hist_axis,forecast_axis),next_10_days['FutureForecast'].tolist(), color = 'orange')

plt.title('LSTM Forecast for Next 10 Days')
plt.xlabel('Hours')
plt.ylabel('Temperature')

# save the figure
plt.savefig('Pics_Models/lstm_univariate_forecast_pic.png')
plt.savefig('Pics_Models/lstm_univariate_forecast_pdf.pdf')

In [6]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Sample data
data = pd.read_csv("TSLA.csv")

# Ensure Date is in datetime format and set it as index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Identify target column and feature columns
target_column = 'Close'
feature_columns = data.columns.tolist()
feature_columns.remove(target_column)

# Scaling features and target column
scalers = {}
for column in data.columns:
    scaler = MinMaxScaler(feature_range=(0, 1))
    data[[column]] = scaler.fit_transform(data[[column]])
    scalers[column] = scaler

# Parameters
n_lookback = 150  # number of past days to look back
n_forecast = 30   # number of days to forecast

# Prepare training data
X = []
Y = []

for i in range(n_lookback, len(data) - n_forecast + 1):
    X.append(data[feature_columns].iloc[i - n_lookback: i].values)
    Y.append(data[target_column].iloc[i: i + n_forecast].values)

X = np.array(X)
Y = np.array(Y)

# Build the model
model = Sequential()
model.add(LSTM(units=32, return_sequences=True, input_shape=(n_lookback, X.shape[2])))
model.add(LSTM(units=32))
model.add(Dense(n_forecast))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, Y, epochs=100, batch_size=8, verbose=0)

# Forecasting
X_ = data[feature_columns].iloc[-n_lookback:].values.reshape(1, n_lookback, len(feature_columns))
Y_ = model.predict(X_).reshape(-1, 1)

# Inverse transform the forecasted data
Y_ = scalers[target_column].inverse_transform(Y_)

# Prepare results
df_past = data[[target_column]].reset_index()
df_past.rename(columns={target_column: 'Actual'}, inplace=True)
df_past['Forecast'] = np.nan
df_past['Forecast'].iloc[-1] = df_past['Actual'].iloc[-1]

df_future = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_future['Date'] = pd.date_range(start=df_past['Date'].iloc[-1] + pd.Timedelta(days=1), periods=n_forecast)
df_future['Forecast'] = Y_.flatten()
df_future['Actual'] = np.nan

results = df_past.append(df_future).set_index('Date')

# Print results
print(results)


KeyboardInterrupt: 

In [11]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Sample data
data = pd.read_csv("TSLA.csv")
data = data.tail(1000)

# Ensure Date is in datetime format and set it as index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Identify target column and feature columns
target_column = 'Close'
feature_columns = data.columns.tolist()
feature_columns.remove(target_column)

# Scaling features and target column
scalers = {}
for column in data.columns:
    scaler = MinMaxScaler(feature_range=(0, 1))
    data[[column]] = scaler.fit_transform(data[[column]])
    scalers[column] = scaler

# Parameters
n_lookback = 50  # number of past days to look back
n_forecast = 30   # number of days to forecast
validation_split = 0.2

# Split data into training and validation sets
train_size = int(len(data) * (1 - validation_split))
data_train = data.iloc[:train_size]
data_valid = data.iloc[train_size:]

print(f'Training data length: {len(data_train)}')
print(f'Validation data length: {len(data_valid)}')

# Prepare training data
X_train = []
Y_train = []

for i in range(n_lookback, len(data_train) - n_forecast + 1):
    X_train.append(data_train[feature_columns].iloc[i - n_lookback: i].values)
    Y_train.append(data_train[target_column].iloc[i: i + n_forecast].values)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

# Prepare validation data
X_valid = []
Y_valid = []

for i in range(n_lookback, len(data_valid) - n_forecast + 1):
    X_valid.append(data_valid[feature_columns].iloc[i - n_lookback: i].values)
    Y_valid.append(data_valid[target_column].iloc[i: i + n_forecast].values)

X_valid = np.array(X_valid)
Y_valid = np.array(Y_valid)

print(f'X_valid shape: {X_valid.shape}')
print(f'Y_valid shape: {Y_valid.shape}')

# Check if the data arrays are not empty
if X_train.size == 0 or Y_train.size == 0 or X_valid.size == 0 or Y_valid.size == 0:
    raise ValueError("Expected input data to be non-empty.")

# Build the model
model = Sequential()
model.add(LSTM(units=32, return_sequences=True, input_shape=(n_lookback, X_train.shape[2])))
model.add(LSTM(units=32))
model.add(Dense(n_forecast))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=100, batch_size=128, verbose=1, validation_data=(X_valid, Y_valid))

# Evaluate the model on the validation set
Y_valid_pred = model.predict(X_valid)
Y_valid_pred = scalers[target_column].inverse_transform(Y_valid_pred.reshape(-1, 1)).reshape(Y_valid_pred.shape)
Y_valid_actual = scalers[target_column].inverse_transform(Y_valid.reshape(-1, 1)).reshape(Y_valid.shape)

validation_rmse = np.sqrt(mean_squared_error(Y_valid_actual, Y_valid_pred))
print(f'Validation RMSE: {validation_rmse}')

# Forecasting using the whole dataset
X_ = data[feature_columns].iloc[-n_lookback:].values.reshape(1, n_lookback, len(feature_columns))
Y_ = model.predict(X_).reshape(-1, 1)

# Inverse transform the forecasted data
Y_ = scalers[target_column].inverse_transform(Y_)

# Prepare results
df_past = data[[target_column]].reset_index()
df_past.rename(columns={target_column: 'Actual'}, inplace=True)
df_past['Forecast'] = np.nan
df_past['Forecast'].iloc[-1] = df_past['Actual'].iloc[-1]

df_future = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_future['Date'] = pd.date_range(start=df_past['Date'].iloc[-1] + pd.Timedelta(days=1), periods=n_forecast)
df_future['Forecast'] = Y_.flatten()
df_future['Actual'] = np.nan

results = df_past.append(df_future).set_index('Date')

# Print results
print(results)


Training data length: 800
Validation data length: 200
X_train shape: (721, 50, 5)
Y_train shape: (721, 30)
X_valid shape: (121, 50, 5)
Y_valid shape: (121, 30)
Epoch 1/100
6/6 [==============================] - 3s 164ms/step - loss: 0.0608 - val_loss: 0.4698
Epoch 2/100
6/6 [==============================] - 0s 42ms/step - loss: 0.0438 - val_loss: 0.3204
Epoch 3/100
6/6 [==============================] - 0s 42ms/step - loss: 0.0271 - val_loss: 0.1903
Epoch 4/100
6/6 [==============================] - 0s 45ms/step - loss: 0.0150 - val_loss: 0.1163
Epoch 5/100
6/6 [==============================] - 0s 45ms/step - loss: 0.0078 - val_loss: 0.0644
Epoch 6/100
6/6 [==============================] - 0s 47ms/step - loss: 0.0048 - val_loss: 0.0503
Epoch 7/100
6/6 [==============================] - 0s 47ms/step - loss: 0.0034 - val_loss: 0.0428
Epoch 8/100
6/6 [==============================] - 0s 51ms/step - loss: 0.0027 - val_loss: 0.0435
Epoch 9/100
6/6 [==============================] - 0s 4

C:\Users\karol\AppData\Local\Temp\ipykernel_1180\904146354.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_past['Forecast'].iloc[-1] = df_past['Actual'].iloc[-1]
C:\Users\karol\AppData\Local\Temp\ipykernel_1180\904146354.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = df_past.append(df_future).set_index('Date')


In [12]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Load and prepare the data
data = pd.read_csv("TSLA.csv")
data = data.tail(1000)

# Ensure Date is in datetime format and set it as index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Identify target column and feature columns
target_column = 'Close'
feature_columns = data.columns.tolist()
feature_columns.remove(target_column)

# Scaling features and target column
scalers = {}
for column in data.columns:
    scaler = MinMaxScaler(feature_range=(0, 1))
    data[[column]] = scaler.fit_transform(data[[column]])
    scalers[column] = scaler

# Parameters
n_lookback = 50  # number of past days to look back
n_forecast = 30   # number of days to forecast
validation_split = 0.2

# Split data into training and validation sets
train_size = int(len(data) * (1 - validation_split))
data_train = data.iloc[:train_size]
data_valid = data.iloc[train_size:]

print(f'Training data length: {len(data_train)}')
print(f'Validation data length: {len(data_valid)}')

# Prepare training data
X_train = []
Y_train = []

for i in range(n_lookback, len(data_train) - n_forecast + 1):
    X_train.append(data_train[feature_columns].iloc[i - n_lookback: i].values)
    Y_train.append(data_train[target_column].iloc[i: i + n_forecast].values)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

# Prepare validation data
X_valid = []
Y_valid = []

for i in range(n_lookback, len(data_valid) - n_forecast + 1):
    X_valid.append(data_valid[feature_columns].iloc[i - n_lookback: i].values)
    Y_valid.append(data_valid[target_column].iloc[i: i + n_forecast].values)

X_valid = np.array(X_valid)
Y_valid = np.array(Y_valid)

print(f'X_valid shape: {X_valid.shape}')
print(f'Y_valid shape: {Y_valid.shape}')

# Check if the data arrays are not empty
if X_train.size == 0 or Y_train.size == 0 or X_valid.size == 0 or Y_valid.size == 0:
    raise ValueError("Expected input data to be non-empty.")

# Build the model
model = Sequential()
model.add(LSTM(units=32, return_sequences=True, input_shape=(n_lookback, X_train.shape[2])))
model.add(LSTM(units=32))
model.add(Dense(n_forecast))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=100, batch_size=128, verbose=1, validation_data=(X_valid, Y_valid))

# Evaluate the model on the validation set
Y_valid_pred = model.predict(X_valid)

# Reshape Y_valid and Y_valid_pred for inverse transform
Y_valid_pred_reshaped = Y_valid_pred.reshape(-1, 1)
Y_valid_reshaped = Y_valid.reshape(-1, 1)

Y_valid_pred_inverse = scalers[target_column].inverse_transform(Y_valid_pred_reshaped).reshape(Y_valid_pred.shape)
Y_valid_actual_inverse = scalers[target_column].inverse_transform(Y_valid_reshaped).reshape(Y_valid.shape)

validation_rmse = np.sqrt(mean_squared_error(Y_valid_actual_inverse, Y_valid_pred_inverse))
print(f'Validation RMSE: {validation_rmse}')

# Forecasting using the whole dataset
X_ = data[feature_columns].iloc[-n_lookback:].values.reshape(1, n_lookback, len(feature_columns))
Y_ = model.predict(X_).reshape(-1, 1)

# Inverse transform the forecasted data
Y_ = scalers[target_column].inverse_transform(Y_)

# Prepare results
df_past = data[[target_column]].reset_index()
df_past.rename(columns={target_column: 'Actual'}, inplace=True)
df_past['Forecast'] = np.nan
df_past['Forecast'].iloc[-1] = df_past['Actual'].iloc[-1]

df_future = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_future['Date'] = pd.date_range(start=df_past['Date'].iloc[-1] + pd.Timedelta(days=1), periods=n_forecast)
df_future['Forecast'] = Y_.flatten()
df_future['Actual'] = np.nan

results = df_past.append(df_future).set_index('Date')

# Print results
print(results)


Training data length: 800
Validation data length: 200
X_train shape: (721, 50, 5)
Y_train shape: (721, 30)
X_valid shape: (121, 50, 5)
Y_valid shape: (121, 30)
Epoch 1/100
6/6 [==============================] - 3s 119ms/step - loss: 0.0616 - val_loss: 0.4857
Epoch 2/100
6/6 [==============================] - 0s 25ms/step - loss: 0.0471 - val_loss: 0.3482
Epoch 3/100
6/6 [==============================] - 0s 26ms/step - loss: 0.0321 - val_loss: 0.2217
Epoch 4/100
6/6 [==============================] - 0s 29ms/step - loss: 0.0183 - val_loss: 0.1377
Epoch 5/100
6/6 [==============================] - 0s 28ms/step - loss: 0.0098 - val_loss: 0.0758
Epoch 6/100
6/6 [==============================] - 0s 25ms/step - loss: 0.0056 - val_loss: 0.0537
Epoch 7/100
6/6 [==============================] - 0s 25ms/step - loss: 0.0036 - val_loss: 0.0451
Epoch 8/100
6/6 [==============================] - 0s 25ms/step - loss: 0.0029 - val_loss: 0.0446
Epoch 9/100
6/6 [==============================] - 0s 2

C:\Users\karol\AppData\Local\Temp\ipykernel_1180\2368592516.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_past['Forecast'].iloc[-1] = df_past['Actual'].iloc[-1]
C:\Users\karol\AppData\Local\Temp\ipykernel_1180\2368592516.py:113: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = df_past.append(df_future).set_index('Date')


In [21]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Sample data
data = pd.read_csv("TSLA.csv")
data = data.tail(1000)

# Ensure Date is in datetime format and set it as index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Identify target column and feature columns
target_column = 'Close'
feature_columns = data.columns.tolist()
feature_columns.remove(target_column)

# Scaling features and target column
scalers = {}
for column in data.columns:
    scaler = MinMaxScaler(feature_range=(0, 1))
    data[[column]] = scaler.fit_transform(data[[column]])
    scalers[column] = scaler

# Parameters
n_lookback = 50  # number of past days to look back
n_forecast = 5   # number of days to forecast
validation_split = 0.2

# Split data into training and validation sets
train_size = int(len(data) * (1 - validation_split))
data_train = data.iloc[:train_size]
data_valid = data.iloc[train_size:]

print(f'Training data length: {len(data_train)}')
print(f'Validation data length: {len(data_valid)}')

# Prepare training data
X_train = []
Y_train = []

for i in range(n_lookback, len(data_train) - n_forecast + 1):
    X_train.append(data_train[feature_columns].iloc[i - n_lookback: i].values)
    Y_train.append(data_train[target_column].iloc[i: i + n_forecast].values)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

# Prepare validation data
X_valid = []
Y_valid = []

for i in range(n_lookback, len(data_valid) - n_forecast + 1):
    X_valid.append(data_valid[feature_columns].iloc[i - n_lookback: i].values)
    Y_valid.append(data_valid[target_column].iloc[i: i + n_forecast].values)

X_valid = np.array(X_valid)
Y_valid = np.array(Y_valid)

print(f'X_valid shape: {X_valid.shape}')
print(f'Y_valid shape: {Y_valid.shape}')

# Check if the data arrays are not empty
if X_train.size == 0 or Y_train.size == 0 or X_valid.size == 0 or Y_valid.size == 0:
    raise ValueError("Expected input data to be non-empty.")

# Build the model
model = Sequential()
model.add(LSTM(units=32, return_sequences=True, input_shape=(n_lookback, X_train.shape[2])))
model.add(LSTM(units=32))
model.add(Dense(n_forecast))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=100, batch_size=128, verbose=1, validation_data=(X_valid, Y_valid))

# Evaluate the model on the validation set
Y_valid_pred = model.predict(X_valid)

# Reshape Y_valid and Y_valid_pred for inverse transform
Y_valid_pred_reshaped = Y_valid_pred.reshape(-1, 1)
Y_valid_reshaped = Y_valid.reshape(-1, 1)

Y_valid_pred_inverse = scalers[target_column].inverse_transform(Y_valid_pred_reshaped).reshape(Y_valid_pred.shape)
Y_valid_actual_inverse = scalers[target_column].inverse_transform(Y_valid_reshaped).reshape(Y_valid.shape)

validation_rmse = np.sqrt(mean_squared_error(Y_valid_actual_inverse, Y_valid_pred_inverse))
print(f'Validation RMSE: {validation_rmse}')

# Forecasting using the whole dataset
X_ = data[feature_columns].iloc[-n_lookback:].values.reshape(1, n_lookback, len(feature_columns))
Y_ = model.predict(X_).reshape(-1, 1)

# Inverse transform the forecasted data
Y_ = scalers[target_column].inverse_transform(Y_)

# Prepare results
df_past = data[[target_column]].reset_index()
df_past.rename(columns={target_column: 'Actual'}, inplace=True)
df_past['Forecast'] = np.nan
df_past['Forecast'].iloc[-1] = df_past['Actual'].iloc[-1]

df_future = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_future['Date'] = pd.date_range(start=df_past['Date'].iloc[-1] + pd.Timedelta(days=1), periods=n_forecast)
df_future['Forecast'] = Y_.flatten()
df_future['Actual'] = np.nan

results = pd.concat([df_past, df_future]).set_index('Date')

# Print results
print(results)




Training data length: 800
Validation data length: 200
X_train shape: (746, 50, 5)
Y_train shape: (746, 5)
X_valid shape: (146, 50, 5)
Y_valid shape: (146, 5)
Epoch 1/100
6/6 [==============================] - 3s 118ms/step - loss: 0.0630 - val_loss: 0.2671
Epoch 2/100
6/6 [==============================] - 0s 27ms/step - loss: 0.0177 - val_loss: 0.0528
Epoch 3/100
6/6 [==============================] - 0s 28ms/step - loss: 0.0069 - val_loss: 0.0187
Epoch 4/100
6/6 [==============================] - 0s 26ms/step - loss: 0.0033 - val_loss: 0.0153
Epoch 5/100
6/6 [==============================] - 0s 27ms/step - loss: 0.0023 - val_loss: 0.0155
Epoch 6/100
6/6 [==============================] - 0s 33ms/step - loss: 0.0020 - val_loss: 0.0125
Epoch 7/100
6/6 [==============================] - 0s 41ms/step - loss: 0.0013 - val_loss: 0.0106
Epoch 8/100
6/6 [==============================] - 0s 26ms/step - loss: 0.0012 - val_loss: 0.0106
Epoch 9/100
6/6 [==============================] - 0s 29m

C:\Users\karol\AppData\Local\Temp\ipykernel_1180\174330883.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_past['Forecast'].iloc[-1] = df_past['Actual'].iloc[-1]


In [28]:
print(data['Close'].scale)

df_past.head(-10)


Date
2018-04-06    0.020154
2018-04-09    0.018539
2018-04-10    0.021058
2018-04-11    0.020427
2018-04-12    0.019280
                ...   
2022-03-18    0.728234
2022-03-21    0.741440
2022-03-22    0.802423
2022-03-23    0.806719
2022-03-24    0.819121
Name: Close, Length: 1000, dtype: float64


,Date,Actual,Forecast
0,2018-04-06,0.020154,NaN
1,2018-04-09,0.018539,NaN
2,2018-04-10,0.021058,NaN
3,2018-04-11,0.020427,NaN
4,2018-04-12,0.019280,NaN
...,...,...,...
985,2022-03-04,0.672042,NaN
986,2022-03-07,0.643812,NaN
987,2022-03-08,0.660410,NaN
988,2022-03-09,0.689360,NaN


In [27]:
df_future.head(-10)

,Date,Actual,Forecast
